In [1]:
import tensorflow as tf
import numpy as np
import sys
sys.path.append("..") # Adds higher directory to python modules path.
import branchingdnn as branching
from branchingdnn.utils import * 
from branchingdnn.profiler import model_profiler as profiler
import h5py
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Activation, Concatenate
from tensorflow.keras.layers import Flatten, Dropout
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D

Using TensorFlow backend.


In [28]:
###normal
inputs = keras.Input(shape=(227,227,3))
x = keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3))(inputs)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)

###compress the rest
# x = keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same")(x)
# x = keras.layers.BatchNormalization()(x)
# x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
fire2_squeeze = Convolution2D(
  16, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
  padding='same', name='fire2_squeeze',
  data_format="channels_last")(x)
fire2_expand1 = Convolution2D(
  64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
  padding='same', name='fire2_expand1',
  data_format="channels_last")(fire2_squeeze)
fire2_expand2 = Convolution2D(
  64, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
  padding='same', name='fire2_expand2',
  data_format="channels_last")(fire2_squeeze)
merge2 = Concatenate(axis=1)([fire2_expand1, fire2_expand2])

fire3_squeeze = Convolution2D(
  16, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
  padding='same', name='fire3_squeeze',
  data_format="channels_last")(merge2)
fire3_expand1 = Convolution2D(
  64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
  padding='same', name='fire3_expand1',
  data_format="channels_last")(fire3_squeeze)
fire3_expand2 = Convolution2D(
  64, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
  padding='same', name='fire3_expand2',
  data_format="channels_last")(fire3_squeeze)
merge3 = Concatenate(axis=1)([fire3_expand1, fire3_expand2])

fire4_squeeze = Convolution2D(
  32, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
  padding='same', name='fire4_squeeze',
  data_format="channels_last")(merge3)
fire4_expand1 = Convolution2D(
  128, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
  padding='same', name='fire4_expand1',
  data_format="channels_last")(fire4_squeeze)
fire4_expand2 = Convolution2D(
  128, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
  padding='same', name='fire4_expand2',
  data_format="channels_last")(fire4_squeeze)
merge4 = Concatenate(axis=1)([fire4_expand1, fire4_expand2])
maxpool4 = MaxPooling2D(
  pool_size=(3, 3), strides=(2, 2), name='maxpool4',
  data_format="channels_last")(merge4)

fire5_squeeze = Convolution2D(
  32, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
  padding='same', name='fire5_squeeze',
  data_format="channels_last")(maxpool4)
fire5_expand1 = Convolution2D(
  128, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
  padding='same', name='fire5_expand1',
  data_format="channels_last")(fire5_squeeze)
fire5_expand2 = Convolution2D(
  128, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
  padding='same', name='fire5_expand2',
  data_format="channels_last")(fire5_squeeze)
merge5 = Concatenate(axis=1)([fire5_expand1, fire5_expand2])

fire6_squeeze = Convolution2D(
  48, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
  padding='same', name='fire6_squeeze',
  data_format="channels_last")(merge5)
fire6_expand1 = Convolution2D(
  192, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
  padding='same', name='fire6_expand1',
  data_format="channels_last")(fire6_squeeze)
fire6_expand2 = Convolution2D(
  192, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
  padding='same', name='fire6_expand2',
  data_format="channels_last")(fire6_squeeze)
merge6 = Concatenate(axis=1)([fire6_expand1, fire6_expand2])

fire7_squeeze = Convolution2D(
  48, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
  padding='same', name='fire7_squeeze',
  data_format="channels_last")(merge6)
fire7_expand1 = Convolution2D(
  192, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
  padding='same', name='fire7_expand1',
  data_format="channels_last")(fire7_squeeze)
fire7_expand2 = Convolution2D(
  192, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
  padding='same', name='fire7_expand2',
  data_format="channels_last")(fire7_squeeze)
merge7 = Concatenate(axis=1)([fire7_expand1, fire7_expand2])

fire8_squeeze = Convolution2D(
  64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
  padding='same', name='fire8_squeeze',
  data_format="channels_last")(merge7)
fire8_expand1 = Convolution2D(
  256, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
  padding='same', name='fire8_expand1',
  data_format="channels_last")(fire8_squeeze)
fire8_expand2 = Convolution2D(
  256, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
  padding='same', name='fire8_expand2',
  data_format="channels_last")(fire8_squeeze)
merge8 = Concatenate(axis=1)([fire8_expand1, fire8_expand2])

maxpool8 = MaxPooling2D(
  pool_size=(3, 3), strides=(2, 2), name='maxpool8',
  data_format="channels_last")(merge8)
fire9_squeeze = Convolution2D(
  64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
  padding='same', name='fire9_squeeze',
  data_format="channels_last")(maxpool8)
fire9_expand1 = Convolution2D(
  256, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
  padding='same', name='fire9_expand1',
  data_format="channels_last")(fire9_squeeze)
fire9_expand2 = Convolution2D(
  256, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
  padding='same', name='fire9_expand2',
  data_format="channels_last")(fire9_squeeze)
merge9 = Concatenate(axis=1)([fire9_expand1, fire9_expand2])

fire9_dropout = Dropout(0.5, name='fire9_dropout')(merge9)
conv10 = Convolution2D(
  10, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
  padding='valid', name='conv10'
  )(fire9_dropout)

global_avgpool10 = GlobalAveragePooling2D()(conv10)
softmax = Activation("softmax", name='softmax')(global_avgpool10)


# x = layers.Flatten(name=tf.compat.v1.get_default_graph().unique_name("branch_flatten"))(x)
# x = layers.Dense(124, activation="relu",name=tf.compat.v1.get_default_graph().unique_name("branch124"))(x)
# x = layers.Dense(64, activation="relu",name=tf.compat.v1.get_default_graph().unique_name("branch64"))(x)
# x = layers.Dense(10, name=tf.compat.v1.get_default_graph().unique_name("branch_output"))(x)
# # output = branch.BranchEndpoint(name=tf.compat.v1.get_default_graph().unique_name("branch_softmax"))(branchLayer, targets, teacher_sm)
# softmax = (layers.Softmax(name=tf.compat.v1.get_default_graph().unique_name("branch_softmax"))(x))



x = keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(4096, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)

# # ### first branch
# # branchLayer = keras.layers.Flatten(name=tf.compat.v1.get_default_graph().unique_name("branch_flatten"))(x)
# # branchLayer = keras.layers.Dense(124, activation="relu",name=tf.compat.v1.get_default_graph().unique_name("branch124"))(branchLayer)
# # branchLayer = keras.layers.Dense(64, activation="relu",name=tf.compat.v1.get_default_graph().unique_name("branch64"))(branchLayer)
# # branchLayer = keras.layers.Dense(10, name=tf.compat.v1.get_default_graph().unique_name("branch_output"))(branchLayer)

x = keras.layers.Dense(4096, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)

# # ### second Branch
# # branchLayer2 = keras.layers.Flatten(name=tf.compat.v1.get_default_graph().unique_name("branch_flatten"))(x)
# # branchLayer2 = keras.layers.Dense(10, name=tf.compat.v1.get_default_graph().unique_name("branch_output"))(branchLayer2)

softmax = keras.layers.Dense(10, activation='softmax')(x)
    
    

In [29]:
# model = keras.Model(inputs=inputs, outputs=[x,branchLayer,branchLayer2], name="alexnet")
model = keras.Model(inputs=inputs, outputs=[softmax], name="alexnet")
#     model.save("models/alexNetv6_compressed.hdf5")
model.summary()


Model: "alexnet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 384)       332160    
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 384)       1536      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 384)       1478

In [30]:

#     checkpoint = keras.callbacks.ModelCheckpoint("models/alexNetv6_compressed.hdf5", monitor='val_loss',verbose=1,save_best_only=True, mode='auto',save_freq=1)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001,momentum=0.9), metrics=['accuracy'])
    model.save("tempModel.hdf5")


In [31]:
tf.keras.backend.clear_session()

profiler.getWholeFlops("tempModel.hdf5")

Model: "alexnet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 384)       332160    
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 384)       1536      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 384)       1478

49 ops no flops stats due to incomplete shapes.
49 ops no flops stats due to incomplete shapes.


Total Flops : 778568609
Enc. Flops : 778568609


###squeeze net

    before flops:        140172
    with branch_1:     34886546
    branch_1 wo flatten: 222098
    branch_1 wi compres:2819003
    model(norm):      778568609
    model(compress)     2819003
    